In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 필요한 라이브러리 임포트
import os
import json
from typing import List
import math
from glob import glob

from tqdm import tqdm
import cv2
import numpy as np
from PIL import Image

In [3]:
CLASS_NAMES_EN = ('small ship', 'large ship', 'civilian aircraft', 'military aircraft',
                  'small car', 'bus', 'truck', 'train', 'crane', 'bridge', 'oil tank',
                  'dam', 'indoor playground', 'outdoor playground', 'helipad', 'roundabout',
                  'helicopter', 'individual container', 'grouped container', 'swimming pool','etc')

In [4]:
def convert_xywha_to_8coords(xywha, is_clockwise=False):
    x, y, w, h, a = xywha
    angle = a if is_clockwise else -a

    lt_x, lt_y = -w / 2, -h / 2
    rt_x, rt_y = w / 2, - h/ 2
    rb_x, rb_y = w / 2, h / 2
    lb_x, lb_y = - w / 2, h / 2

    lt_x_ = lt_x * math.cos(angle) - lt_y * math.sin(angle) + x
    lt_y_ = lt_x * math.sin(angle) + lt_y * math.cos(angle) + y
    rt_x_ = rt_x * math.cos(angle) - rt_y * math.sin(angle) + x
    rt_y_ = rt_x * math.sin(angle) + rt_y * math.cos(angle) + y
    lb_x_ = lb_x * math.cos(angle) - lb_y * math.sin(angle) + x
    lb_y_ = lb_x * math.sin(angle) + lb_y * math.cos(angle) + y
    rb_x_ = rb_x * math.cos(angle) - rb_y * math.sin(angle) + x
    rb_y_ = rb_x * math.sin(angle) + rb_y * math.cos(angle) + y

    return [lt_x_, lt_y_, rt_x_, rt_y_, rb_x_, rb_y_, lb_x_, lb_y_]


In [5]:
def convert_8coords_to_4coords(coords):
    x_coords = coords[0::2]
    y_coords = coords[1::2]
    
    xmin = min(x_coords)
    ymin = min(y_coords)

    xmax = max(x_coords)
    ymax = max(y_coords)

    w = xmax-xmin
    h = ymax-ymin

    return [xmin, ymin, w, h]

In [6]:
def convert_labels_to_objects(coords, class_ids, class_names, image_ids, difficult=0, is_clockwise=False):
    objs = list()
    inst_count = 1

    for polygons, cls_id, cls_name, img_id in tqdm(zip(coords, class_ids, class_names, image_ids), desc="converting labels to objects"):
        xmin, ymin, w, h = convert_8coords_to_4coords(polygons)
        single_obj = {}
        single_obj['difficult'] = difficult
        single_obj['area'] = w*h
        if cls_name in CLASS_NAMES_EN:
            single_obj['category_id'] = CLASS_NAMES_EN.index(cls_name) + 1
        else:
            continue
        single_obj['segmentation'] = [[int(p) for p in polygons]]
        single_obj['iscrowd'] = 0
        single_obj['bbox'] = (xmin, ymin, w, h)
        single_obj['image_id'] = img_id
        single_obj['id'] = inst_count
        inst_count += 1
        objs.append(single_obj)

    print('objects', len(objs))
    return objs

In [7]:
def load_geojsons(filepath):
    """ Gets label data from a geojson label file

    :param (str) filename: file path to a geojson label file
    :return: (numpy.ndarray, numpy.ndarray ,numpy.ndarray) coords, chips, and classes corresponding to
            the coordinates, image names, and class codes for each ground truth.
    """
    jsons = glob(os.path.join(filepath, '*.json'))
    features = []
    for json_path in tqdm(jsons, desc='loading geojson files'):
        with open(json_path) as f:
            data_dict = json.load(f)
        features += data_dict['features']

    obj_coords = list()
    image_ids = list()
    class_indices = list()
    class_names = list()

    for feature in tqdm(features, desc='extracting features'):
        properties = feature['properties']
        image_ids.append(properties['image_id'].replace('PS4', 'PS3')[:-4]+'.png')
        obj_coords.append([float(num) for num in properties['object_imcoords'].split(",")])
        class_indices.append(properties['type_id'])
        class_names.append(properties['type_name'])

    return image_ids, obj_coords, class_indices, class_names

In [8]:
def geojson2coco(imageroot: str, geojsonpath: str, destfile, difficult='-1'):
    # set difficult to filter '2', '1', or do not filter, set '-1'

    data_dict = {}
    data_dict['images'] = []
    data_dict['categories'] = []
    data_dict['annotations'] = []
    for idex, name in enumerate(CLASS_NAMES_EN):
        single_cat = {'id': idex + 1, 'name': name, 'supercategory': name}
        data_dict['categories'].append(single_cat)

    inst_count = 1
    image_id = 1
    with open(destfile, 'w') as f_out:
        img_files, obj_coords, cls_ids, class_names = load_geojsons(geojsonpath)
        img_id_map= {img_file:i+1 for i, img_file in enumerate(list(set(img_files)))}
        image_ids = [img_id_map[img_file] for img_file in img_files]
        objs = convert_labels_to_objects(obj_coords, cls_ids, class_names, image_ids, difficult=difficult, is_clockwise=False)
        data_dict['annotations'].extend(objs)

        for imgfile in tqdm(img_id_map, desc='saving img info'):
            imagepath = os.path.join(imageroot, imgfile)
            img_id = img_id_map[imgfile]
            img = cv2.imread(imagepath)
            height, width, c = img.shape
            single_image = {}
            single_image['file_name'] = imgfile
            single_image['id'] = img_id
            single_image['width'] = width
            single_image['height'] = height
            data_dict['images'].append(single_image)

        json.dump(data_dict, f_out)

In [13]:
rootfolder = '/content/drive/MyDrive/sia/data/Objects/'

geojson2coco(imageroot=os.path.join(rootfolder, 'train/images'),
                 geojsonpath=os.path.join(rootfolder, 'label'),
                 destfile=os.path.join(rootfolder, 'traincoco.json'))

extracting features: 100%|██████████| 423750/423750 [00:03<00:00, 123635.77it/s]
converting labels to objects: 423750it [00:03, 138528.97it/s]


objects 423750


saving img info: 100%|██████████| 1748/1748 [13:36<00:00,  2.14it/s]


In [14]:
geojson2coco(imageroot=os.path.join(rootfolder, 'val/images'),
                 geojsonpath=os.path.join(rootfolder, 'val/label'),
                 destfile=os.path.join(rootfolder, 'valcoco.json'))

extracting features: 100%|██████████| 51208/51208 [00:00<00:00, 133318.30it/s]
converting labels to objects: 51208it [00:00, 92425.08it/s]


objects 51208


saving img info: 100%|██████████| 220/220 [01:33<00:00,  2.35it/s]


In [12]:
rootfolder = '/content/drive/'

geojson2coco(imageroot=os.path.join(rootfolder, 'Shareddrives/OD/workspace/data//test/images'),
                 geojsonpath=os.path.join(rootfolder, 'Shareddrives/OD/workspace/data//test_etc/labels'),
                 destfile=os.path.join(rootfolder, 'MyDrive/sia/data/Objects/testcoco.json'))

extracting features: 100%|██████████| 6350/6350 [00:00<00:00, 148577.37it/s]
converting labels to objects: 6350it [00:00, 66118.11it/s]


objects 6350


saving img info: 100%|██████████| 22/22 [00:13<00:00,  1.64it/s]


In [ ]:
import json

label_path = '/content/drive/MyDrive/sia/data/Objects/val/valcoco.json'

# 단순히 파일 열어서 변수에 저장
with open(label_path, 'r') as f:

    json_data = json.load(f)

In [ ]:
print(json.dumps(json_data, indent='\t'))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
json_data

{'annotations': [{'area': 46.788056950187546,
   'bbox': [417.7663494408794,
    722.8766877442824,
    8.663940605675066,
    5.400320602329657],
   'category_id': 4,
   'difficult': '-1',
   'id': 1,
   'image_id': 176,
   'iscrowd': 0,
   'segmentation': [[418, 728, 417, 724, 425, 722, 426, 727]]},
  {'area': 20.49405344915371,
   'bbox': [787.8653232316502,
    685.0905628660045,
    5.160927236103362,
    3.9710022078566],
   'category_id': 4,
   'difficult': '-1',
   'id': 2,
   'image_id': 176,
   'iscrowd': 0,
   'segmentation': [[792, 685, 793, 688, 788, 689, 787, 685]]},
  {'area': 50.27262692904475,
   'bbox': [270.57406441472284,
    751.0944678705793,
    8.717979037978807,
    5.766545974707924],
   'category_id': 4,
   'difficult': '-1',
   'id': 3,
   'image_id': 176,
   'iscrowd': 0,
   'segmentation': [[271, 756, 270, 752, 278, 751, 279, 755]]},
  {'area': 41.14106983374825,
   'bbox': [237.93983134772853,
    751.8232020410692,
    8.57503184930465,
    4.79777458052

In [ ]:
# images(png), categories, annotations
json_data.keys()

dict_keys(['images', 'categories', 'annotations'])

In [ ]:
json_data_images = json_data['images']
print(len(json_data_images))
json_data_images

220


[{'file_name': 'OBJ03875_PS3_K3A_NIA0163.png',
  'height': 1024,
  'id': 1,
  'width': 1024},
 {'file_name': 'OBJ05188_PS3_K3A_NIA0340.png',
  'height': 1024,
  'id': 2,
  'width': 1024},
 {'file_name': 'OBJ03567_PS3_K3A_NIA0150.png',
  'height': 1024,
  'id': 3,
  'width': 1024},
 {'file_name': 'OBJ03819_PS3_K3A_NIA0159.png',
  'height': 1024,
  'id': 4,
  'width': 1024},
 {'file_name': 'OBJ05183_PS3_K3A_NIA0339.png',
  'height': 1024,
  'id': 5,
  'width': 1024},
 {'file_name': 'OBJ05437_PS3_K3A_NIA0165.png',
  'height': 1024,
  'id': 6,
  'width': 1024},
 {'file_name': 'OBJ03491_PS3_K3A_NIA0144.png',
  'height': 1024,
  'id': 7,
  'width': 1024},
 {'file_name': 'OBJ04778_PS3_K3A_NIA0297.png',
  'height': 1024,
  'id': 8,
  'width': 1024},
 {'file_name': 'OBJ04812_PS3_K3A_NIA0315.png',
  'height': 1024,
  'id': 9,
  'width': 1024},
 {'file_name': 'OBJ03370_PS3_K3A_NIA0137.png',
  'height': 1024,
  'id': 10,
  'width': 1024},
 {'file_name': 'OBJ04807_PS3_K3A_NIA0315.png',
  'height': 

In [ ]:
json_data_categories = json_data['categories']
print(len(json_data_categories))
json_data_categories

21


[{'id': 1, 'name': 'small ship', 'supercategory': 'small ship'},
 {'id': 2, 'name': 'large ship', 'supercategory': 'large ship'},
 {'id': 3, 'name': 'civilian aircraft', 'supercategory': 'civilian aircraft'},
 {'id': 4, 'name': 'military aircraft', 'supercategory': 'military aircraft'},
 {'id': 5, 'name': 'small car', 'supercategory': 'small car'},
 {'id': 6, 'name': 'bus', 'supercategory': 'bus'},
 {'id': 7, 'name': 'truck', 'supercategory': 'truck'},
 {'id': 8, 'name': 'train', 'supercategory': 'train'},
 {'id': 9, 'name': 'crane', 'supercategory': 'crane'},
 {'id': 10, 'name': 'bridge', 'supercategory': 'bridge'},
 {'id': 11, 'name': 'oil tank', 'supercategory': 'oil tank'},
 {'id': 12, 'name': 'dam', 'supercategory': 'dam'},
 {'id': 13, 'name': 'indoor playground', 'supercategory': 'indoor playground'},
 {'id': 14,
  'name': 'outdoor playground',
  'supercategory': 'outdoor playground'},
 {'id': 15, 'name': 'helipad', 'supercategory': 'helipad'},
 {'id': 16, 'name': 'roundabout', '

In [ ]:
json_data_annotations = json_data['annotations']
print(len(json_data_annotations), end='\n')
json_data_annotations

51208


[{'area': 46.788056950187546,
  'bbox': [417.7663494408794,
   722.8766877442824,
   8.663940605675066,
   5.400320602329657],
  'category_id': 4,
  'difficult': '-1',
  'id': 1,
  'image_id': 176,
  'iscrowd': 0,
  'segmentation': [[418, 728, 417, 724, 425, 722, 426, 727]]},
 {'area': 20.49405344915371,
  'bbox': [787.8653232316502,
   685.0905628660045,
   5.160927236103362,
   3.9710022078566],
  'category_id': 4,
  'difficult': '-1',
  'id': 2,
  'image_id': 176,
  'iscrowd': 0,
  'segmentation': [[792, 685, 793, 688, 788, 689, 787, 685]]},
 {'area': 50.27262692904475,
  'bbox': [270.57406441472284,
   751.0944678705793,
   8.717979037978807,
   5.766545974707924],
  'category_id': 4,
  'difficult': '-1',
  'id': 3,
  'image_id': 176,
  'iscrowd': 0,
  'segmentation': [[271, 756, 270, 752, 278, 751, 279, 755]]},
 {'area': 41.14106983374825,
  'bbox': [237.93983134772853,
   751.8232020410692,
   8.57503184930465,
   4.797774580520581],
  'category_id': 4,
  'difficult': '-1',
  'id

In [ ]:
import json

label_path = '/content/drive/Shareddrives/OD/workspace/data/test/testcoco.json'

# 단순히 파일 열어서 변수에 저장
with open(label_path, 'r') as f:

    json_data = json.load(f)

In [ ]:
json_data_categories = json_data['categories']
print(len(json_data_categories))
json_data_categories

21


[{'id': 1, 'name': 'small ship', 'supercategory': 'small ship'},
 {'id': 2, 'name': 'large ship', 'supercategory': 'large ship'},
 {'id': 3, 'name': 'civilian aircraft', 'supercategory': 'civilian aircraft'},
 {'id': 4, 'name': 'military aircraft', 'supercategory': 'military aircraft'},
 {'id': 5, 'name': 'small car', 'supercategory': 'small car'},
 {'id': 6, 'name': 'bus', 'supercategory': 'bus'},
 {'id': 7, 'name': 'truck', 'supercategory': 'truck'},
 {'id': 8, 'name': 'train', 'supercategory': 'train'},
 {'id': 9, 'name': 'crane', 'supercategory': 'crane'},
 {'id': 10, 'name': 'bridge', 'supercategory': 'bridge'},
 {'id': 11, 'name': 'oil tank', 'supercategory': 'oil tank'},
 {'id': 12, 'name': 'dam', 'supercategory': 'dam'},
 {'id': 13, 'name': 'indoor playground', 'supercategory': 'indoor playground'},
 {'id': 14,
  'name': 'outdoor playground',
  'supercategory': 'outdoor playground'},
 {'id': 15, 'name': 'helipad', 'supercategory': 'helipad'},
 {'id': 16, 'name': 'roundabout', '

In [ ]:
import json

label_path = '/content/drive/MyDrive/sia/data/Objects/train/traincoco.json'

# 단순히 파일 열어서 변수에 저장
with open(label_path, 'r') as f:

    json_data = json.load(f)

In [ ]:
json_data_categories = json_data['categories']
print(len(json_data_categories))
json_data_categories

21


[{'id': 1, 'name': 'small ship', 'supercategory': 'small ship'},
 {'id': 2, 'name': 'large ship', 'supercategory': 'large ship'},
 {'id': 3, 'name': 'civilian aircraft', 'supercategory': 'civilian aircraft'},
 {'id': 4, 'name': 'military aircraft', 'supercategory': 'military aircraft'},
 {'id': 5, 'name': 'small car', 'supercategory': 'small car'},
 {'id': 6, 'name': 'bus', 'supercategory': 'bus'},
 {'id': 7, 'name': 'truck', 'supercategory': 'truck'},
 {'id': 8, 'name': 'train', 'supercategory': 'train'},
 {'id': 9, 'name': 'crane', 'supercategory': 'crane'},
 {'id': 10, 'name': 'bridge', 'supercategory': 'bridge'},
 {'id': 11, 'name': 'oil tank', 'supercategory': 'oil tank'},
 {'id': 12, 'name': 'dam', 'supercategory': 'dam'},
 {'id': 13, 'name': 'indoor playground', 'supercategory': 'indoor playground'},
 {'id': 14,
  'name': 'outdoor playground',
  'supercategory': 'outdoor playground'},
 {'id': 15, 'name': 'helipad', 'supercategory': 'helipad'},
 {'id': 16, 'name': 'roundabout', '